## Automation with selenium
[Video](https://youtu.be/f7LEWxX4AVI)

# om Web scraping
https://towardsdatascience.com/data-science-skills-web-scraping-javascript-using-python-97a29738353f

##**Enligt föregående artikel**:
##Using selenium with Firefox web driver

In [1]:
#!apt install chromium-chromedriver
!pip install selenium
!pip install tabulate


In [2]:
import pandas as  pd 
import requests
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from tabulate import tabulate

import urllib.request
from bs4 import BeautifulSoup
import time


# Load all the links with races 

In [4]:
# specify the url

urlpage = "https://www.atg.se/videoarkiv/trav?gameTypes=V75&page="
print(urlpage)
# run firefox webdriver from executable path of your choice
driver = webdriver.Chrome(executable_path='C:\\Users\peter\\Documents\\MyProjects\\gecko\\chromedriver.exe')
#####
# get web pages
data = []
for i in range(1,51):
    driver.get(urlpage+str(i))
    print(urlpage+str(i))
    time.sleep(5)
    results = driver.find_elements_by_xpath("//*[@class='date-col']")
    print('Number of results', len(results))
    # loop over results
    for result in results:
        #product_name = result.text
        link = result.find_element_by_tag_name('a')
        lopp_link = link.get_attribute("href")
        # append dict to array  
        #data.append({"product" : product_name, "link" : product_link})
        data.append(lopp_link)
        

# execute script to scroll down the page
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
# sleep for 30s



https://www.atg.se/videoarkiv/trav?gameTypes=V75&page=
https://www.atg.se/videoarkiv/trav?gameTypes=V75&page=1
Number of results 50
https://www.atg.se/videoarkiv/trav?gameTypes=V75&page=2
Number of results 50
https://www.atg.se/videoarkiv/trav?gameTypes=V75&page=3
Number of results 50
https://www.atg.se/videoarkiv/trav?gameTypes=V75&page=4
Number of results 50


# Start getting all the data

In [ ]:
# get the webdriver for driver2 
driver2 = webdriver.Chrome(executable_path='C:\\Users\peter\\Documents\\MyProjects\\gecko\\chromedriver.exe')

In [ ]:
#load the links
lopp_df = pd.read_csv('C:\\Users\\peter\Documents\\MyProjects\\PyProj\\Trav\\alla_lopp.csv' )

# create empty array to store the ata
komplett=[]


In [ ]:
def kom_till_start(i):
    pth='//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[6]/div/div/table/tbody/tr['+str(i)+']/td[1]'
    kom = driver2.find_elements_by_xpath(pth)
    
    if kom[0].text == 's':
        return False
    if kom[0].text == 'utg':
        return False

    return True

def getSpår(i):
    
    pth='table > tbody > tr:nth-child('+str(i)+') > td.postPositionAndDistance-col'

    sp = driver2.find_element_by_css_selector(pth) 

    txt = sp.text.split()
    spr = None
    dis = None

    if len(txt) > 0:
        dis = txt[0]
    
    if len(txt) == 3:
        spr = txt[2]


    return spr,dis

def getVodds(i):
    pth = 'table > tbody > tr:nth-child('+str(i)+') > td.vOdds-col'
    
    vo = driver2.find_element_by_css_selector(pth)

    return vo.text    

def getPodds(i):
    pth='table > tbody > tr:nth-child('+str(i)+') > td.pOdds-col'
         
    po = driver2.find_element_by_css_selector(pth)
    
    return po.text    

def getPlac(i):

    pth = 'table > tbody > tr:nth-child('+str(i)+') > td.place-col'
    
    pl = driver2.find_element_by_css_selector(pth)

    return pl.text    


In [ ]:
def getHist():
    history=[]
    try:
        for child in range(1,6):
            plac = 'table > tbody > tr:nth-child(' +str(child)+ ') > td.place-col > span'
            dist = 'table > tbody > tr:nth-child(' +str(child)+ ') > td.distance-col'
            spår = 'table > tbody > tr:nth-child(' +str(child)+ ') > td.position-col'
            odds = 'table > tbody > tr:nth-child(' +str(child)+ ') > td.odds-col'
            
            plac = driver2.find_element_by_css_selector(plac)
            dist = driver2.find_element_by_css_selector(dist)
            spår = driver2.find_element_by_css_selector(spår)
            odds = driver2.find_element_by_css_selector(odds)
            history.append([plac.text, dist.text, spår.text, odds.text])
    except:
        logging.warning("except")
        return history

    return history

In [ ]:
# Kör denna först och sätt fullscreen samt klick på anpassa (tick p-odds och distans/spår)
##### AUTOMATISERA DETTA 
driver2.get(lopp_df.Links[0])

## Loop through the links
Open each page with a race and go through all the horses and history


In [102]:

import logging
logging.basicConfig(filename='app.log', filemode='w', format='%(name)s - %(message)s',level=logging.INFO)
logging.warning('STARTING AGAIN FROM ' + testLopp)

# loopa fram till lopp
goon=True  ### False om jag vill starta från testLopp

for lopp in lopp_df.Links:
    if goon==False:
        if lopp != testLopp:
            continue
        goon=True    

    logging.warning(lopp)
    dat = lopp.split('spel/')[1][0:10]  #gräv ut datum
    
    # open lopp web page
    driver2.get(lopp)
    results2d = driver2.find_elements_by_xpath("//*[@class='game-table']//*[@class='horse-name css-1h5vlcx-horseview-styles--horseName']")
    d = driver2.find_elements_by_xpath('//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[6]/div/div/div[1]/div[1]/div[1]/div/div[2]/div[2]/div[2]')
    split1 = d[0].text.split()[0]   # lopp distans (använd hästens distans ist)
    split2 = d[0].text.split()[1]   # start
    i=1
    for result2d in results2d:
        if  kom_till_start(i):
            namn = result2d.text
            plac = getPlac(i)
            #dist = split1
            start = split2
            spår,dist = getSpår(i)
            vodds = getVodds(i)
            podds = getPodds(i)
            horse = [dat, namn, plac, dist, start, spår, vodds, podds]

            result2d.click()  #for history
            time.sleep(1)
            hist = sum(getHist(),[])
            komplett.append(horse+hist)
            #print('history:',history)
            result2d.click()
        else:
            logging.warning(result2d.text+' kom ej')
        i=i+1    


In [103]:
lopp

'https://www.atg.se/spel/2015-11-07/vinnare/eskilstuna/lopp11/resultat'

In [104]:

len(komplett)

27623

In [84]:

# Om jag fortsätta där det bröt
testLopp=lopp
testLopp

'https://www.atg.se/spel/2018-08-26/vinnare/charlottenlund/lopp11/resultat'

In [105]:
#kopia=komplett
df=pd.DataFrame(komplett)
df.shape
df.columns=['datum','häst', 'plac', 'dist', 'start', 'spår', 'vodds', 'podds' , 
'h1_plac', 'h1_dist', 'h1_spår', 'h1_odds',
'h2_plac', 'h2_dist', 'h2_spår', 'h2_odds',
'h3_plac', 'h3_dist', 'h3_spår', 'h3_odds',
'h4_plac', 'h4_dist', 'h4_spår', 'h4_odds',
'h5_plac', 'h5_dist', 'h5_spår', 'h5_odds']
df.head(10)

,datum,häst,plac,dist,start,spår,vodds,podds,h1_plac,h1_dist,...,h3_spår,h3_odds,h4_plac,h4_dist,h4_spår,h4_odds,h5_plac,h5_dist,h5_spår,h5_odds
0,2020-04-04,GIANT SHADOW,1:a,2640,VOLTSTART,7,"1,94","1,34",4,2140,...,2,"1,41",1,1640,8,"1,65",1,1640,11,"1,72"
1,2020-04-04,NERO MAXIMUS,2,2640,VOLTSTART,5,"7,07","2,03",1,2140,...,2,"2,12",3,3160,1,"5,02",3,2140,8,"3,21"
2,2020-04-04,GLOBAL UNSPOKED,3,2640,VOLTSTART,3,"28,73","2,97",0,3180,...,3,"24,02",2,2640,1,"3,83",0,2640,10,"23,03"
3,2020-04-04,GANYBOY,4,2640,VOLTSTART,10,"37,62","3,80 - 7,38",4,2140,...,,"6,40",1,2100,,"3,20",2,2100,,"7,50"
4,2020-04-04,ELDORADO MEARAS,5,2640,VOLTSTART,11,"57,79","5,09 - 10,14",0,2140,...,9,"41,48",3,1640,1,"15,94",1,2140,10,"9,72"
5,2020-04-04,NEELIX,6,2640,VOLTSTART,6,"5,21","1,62 - 2,62",1,3160,...,7,"4,07",1,2140,2,"3,25",5,2140,8,"32,32"
6,2020-04-04,CROSSFIRE SIMONI,7,2640,VOLTSTART,1,"80,04","4,04 - 7,91",4,1640,...,7,"9,55",2,2140,11,"45,98",4,1640,11,"52,37"
7,2020-04-04,MR JÄGER,0,2640,VOLTSTART,8,"87,98","5,87 - 11,76",0,2180,...,3,"14,84",3,1640,6,"14,97",3,2680,6,"55,23"
8,2020-04-04,MAGIC HAPPENS,0,2640,VOLTSTART,12,"39,75","3,85 - 7,48",5,2640,...,,"3,00",1,2640,1,"4,04",2,2140,5,"4,58"
9,2020-04-04,VIKINGS PREACHER,d,2640,VOLTSTART,2,"18,10","3,21 - 6,13",None,None,...,None,None,None,None,None,None,None,None,None,None


In [107]:
df.tail()

,datum,häst,plac,dist,start,spår,vodds,podds,h1_plac,h1_dist,...,h3_spår,h3_odds,h4_plac,h4_dist,h4_spår,h4_odds,h5_plac,h5_dist,h5_spår,h5_odds
27618,2015-11-07,USAIN COLT,0,2140,AUTOSTART,10,"21,84","3,60 - 6,09",6,2140,...,1,"1,47",2,2140,9,"5,09",0,2640,4,"64,94"
27619,2015-11-07,BRITELITE AM,0,2140,AUTOSTART,6,"9,41","2,04 - 3,20",1,2140,...,10,"8,02",6,2140,10,"5,76",6,2140,9,"15,16"
27620,2015-11-07,GLOBAL RACECOURSE,d,2140,AUTOSTART,11,"2,37","1,34 - 1,50",None,None,...,None,None,None,None,None,None,None,None,None,None
27621,2015-11-07,STARO HEY JOE,d,2140,AUTOSTART,4,"13,80","2,46 - 3,99",None,None,...,None,None,None,None,None,None,None,None,None,None
27622,2015-11-07,PERSOS BOJJE,d,2140,AUTOSTART,2,"23,11","3,73 - 6,33",6,2140,...,15,"3,28",0,2140,3,"12,74",1,2640,1,"4,32"


In [106]:
print(df.shape)
df.to_csv('C:\\Users\\peter\Documents\\MyProjects\\PyProj\\Trav\\komplett.csv', index=False)


(27623, 28)


## När allt är klart

In [109]:
driver2.quit()